# Car Price Prediction: Model Comparison
This notebook compares three supervised regression algorithms **Multiple Linear Regression**, **Decision Tree**, and **Gradient Boosting Regressor** on the `Auto Price` dataset.  
All models predict **car prices** using numerical, categorical, and engineered features, and are evaluated with **R²**, **RMSE**, **MAE**, and **5-fold Cross-Validation R²**.


## Step 1: Import Required Libraries


In [135]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from joblib import parallel_backend
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.inspection import permutation_importance
import inspect

## Step 2: Load and Prepare the Dataset


In [138]:
df = pd.read_csv("auto_price_cleaned.csv")
df = df.loc[:, ~df.columns.str.contains("^Unnamed", na=False)]

TARGET = "Price"
y = df[TARGET]
X = df.drop(columns=[TARGET, "Comfort_Convenience"], errors="ignore")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cv5 = KFold(n_splits=5, shuffle=True, random_state=42)


## Step 3: Define Safe RMSE Function and One-Hot Encoder Setup

In [141]:
try:
    from sklearn.metrics import root_mean_squared_error
    def rmse_func(y_true, y_pred): return root_mean_squared_error(y_true, y_pred)
except ImportError:
    def rmse_func(y_true, y_pred): return mean_squared_error(y_true, y_pred, squared=False)

if "sparse_output" in inspect.signature(OneHotEncoder).parameters:
    OHE_KW = dict(drop="first", handle_unknown="ignore", sparse_output=False)
else:
    OHE_KW = dict(drop="first", handle_unknown="ignore", sparse=False)

## Step 4: Feature Engineering
Two transformers are defined:
- **FeatureEngineeringTransformer**: Adds domain-driven features such as Power-to-Weight ratio, Age×Mileage interaction, and Engine Power Index.
- **FeatureEngineeringTransformer_GBR**: Extends the above for Gradient Boosting by adding additional derived variables (e.g., HP_per_Consumption, Mileage_per_Year).
These enhance model interpretability and predictive strength.


In [144]:
class FeatureEngineeringTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        X = X.copy()
        if all(c in X.columns for c in ["Horsepower", "Weight"]):
            X["Power_to_Weight"] = X["Horsepower"] / (X["Weight"] + 1e-6)
        if all(c in X.columns for c in ["Mileage", "Age"]):
            X["Mileage_per_Age"] = X["Mileage"] / (X["Age"] + 1)
            X["Age*Mileage"] = X["Mileage"] * X["Age"]
        if all(c in X.columns for c in ["Horsepower", "Displacement"]):
            X["Engine_Power_Index"] = X["Horsepower"] * X["Displacement"]
        if "Age" in X.columns and "Mileage" in X.columns:
            X["Depreciation_Rate"] = (X["Age"] + 1) / (X["Mileage"] + 1)
        X.replace([np.inf, -np.inf], np.nan, inplace=True)
        X.fillna(0, inplace=True)
        return X

class FeatureEngineeringTransformer_GBR(FeatureEngineeringTransformer):
    def transform(self, X):
        X = super().transform(X.copy())
        if "Horsepower" in X.columns and "Cons_Comb" in X.columns:
            X["HP_per_Consumption"] = X["Horsepower"] / (X["Cons_Comb"] + 1e-3)
        if "Mileage" in X.columns and "Age" in X.columns:
            X["Mileage_per_Year"] = X["Mileage"] / (X["Age"] + 1)
        return X

## Step 5: Interaction Transformer for Linear Regression
This transformer creates pairwise interaction terms between continuous predictors like Horsepower, Weight, Age, and Mileage.  
It helps the Linear Regression model capture mild nonlinear relationships.


In [147]:
class InteractionTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, cols, degree=2, interaction_only=True):
        self.cols = cols
        self.degree = degree
        self.interaction_only = interaction_only
    def fit(self, X, y=None):
        self.cols_ = [c for c in self.cols if c in X.columns]
        if self.cols_:
            self.poly_ = PolynomialFeatures(degree=self.degree, interaction_only=self.interaction_only, include_bias=False).fit(X[self.cols_])
            self.feature_names_ = self.poly_.get_feature_names_out(self.cols_)
        else:
            self.feature_names_ = np.array([])
        return self
    def transform(self, X):
        if not self.cols_:
            return np.empty((len(X), 0))
        return self.poly_.transform(X[self.cols_])
    def get_feature_names_out(self, input_features=None): return self.feature_names_


## Step 6: Model 1 — Multiple Linear Regression
A preprocessing pipeline applies:
- Standard scaling for numeric variables,
- One-Hot Encoding for categorical variables,
- Interaction feature generation.
The model is trained and evaluated using 5-fold cross-validation and tested on the holdout set.
Performance metrics (**R², RMSE, MAE**) are computed in real price units, and the top 15 features are identified using permutation importance.


In [150]:
num_cols_all = X.select_dtypes(exclude="object").columns.tolist()
cat_cols_all = X.select_dtypes(include="object").columns.tolist()
interaction_cols = [c for c in ["Horsepower","Weight","Age","Mileage","Displacement","Cons_Comb"] if c in X.columns]

pre_mlr = ColumnTransformer([
    ("num", StandardScaler(), num_cols_all),
    ("cat", OneHotEncoder(**OHE_KW), cat_cols_all),
    ("inter", InteractionTransformer(interaction_cols), num_cols_all + cat_cols_all)
], remainder="drop", verbose_feature_names_out=False)

pipe_mlr = Pipeline([
    ("preprocess", pre_mlr),
    ("regressor", LinearRegression())
])

cv_scores_mlr = cross_val_score(pipe_mlr, X_train, y_train, cv=cv5, scoring="r2", n_jobs=-1)
pipe_mlr.fit(X_train, y_train)
y_pred_mlr = pipe_mlr.predict(X_test)
R2_mlr, RMSE_mlr, MAE_mlr = r2_score(y_test, y_pred_mlr), rmse_func(y_test, y_pred_mlr), mean_absolute_error(y_test, y_pred_mlr)

perm_mlr = permutation_importance(pipe_mlr, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1, scoring="r2")
fn_num = np.array(num_cols_all)
fn_cat = pipe_mlr.named_steps["preprocess"].named_transformers_["cat"].get_feature_names_out(cat_cols_all) if cat_cols_all else np.array([])
fn_inter = pipe_mlr.named_steps["preprocess"].named_transformers_["inter"].get_feature_names_out()
fn_mlr = np.concatenate([fn_num, fn_cat, fn_inter])
imp_mlr = pd.DataFrame({"Feature": fn_mlr[:len(perm_mlr.importances_mean)], "Importance": perm_mlr.importances_mean}).sort_values("Importance", ascending=False).head(15)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


## Step 7: Model 2 — Decision Tree Regressor
A Decision Tree is trained using a pipeline that integrates feature engineering and one-hot encoding.  
GridSearchCV is used to tune hyperparameters such as `max_depth`, `min_samples_split`, and `max_features`.  
Performance is evaluated on test data and cross-validated on the training data.
Feature importances (top 15) are visualized to show which predictors drive price variance.


In [111]:
pre_dt = ColumnTransformer([("cat", OneHotEncoder(**OHE_KW), selector(dtype_include=object))],
                           remainder="passthrough", verbose_feature_names_out=False)

pipe_dt = Pipeline([
    ("feature_engineering", FeatureEngineeringTransformer()),
    ("preprocess", pre_dt),
    ("model", DecisionTreeRegressor(random_state=42))
])

param_dt = {"model__max_depth":[8,10,12,15,None],"model__min_samples_split":[2,5,10],"model__min_samples_leaf":[1,2,4],"model__max_features":[None,"sqrt","log2"]}
grid_dt = GridSearchCV(pipe_dt, param_grid=param_dt, cv=cv5, scoring="r2", n_jobs=-1)
grid_dt.fit(X_train, y_train)
best_dt = grid_dt.best_estimator_
cv_scores_dt = cross_val_score(best_dt, X_train, y_train, cv=cv5, scoring="r2", n_jobs=-1)

y_pred_dt = best_dt.predict(X_test)
R2_dt, RMSE_dt, MAE_dt = r2_score(y_test, y_pred_dt), rmse_func(y_test, y_pred_dt), mean_absolute_error(y_test, y_pred_dt)
fn_dt = best_dt.named_steps["preprocess"].get_feature_names_out()
imp_dt = pd.DataFrame({"Feature": fn_dt, "Importance": best_dt.named_steps["model"].feature_importances_}).sort_values("Importance", ascending=False).head(15)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning:

## Step 8: Model 3 — Gradient Boosting Regressor
A Gradient Boosting model (ensemble of decision trees) is trained using RandomizedSearchCV for efficiency.  
This model typically achieves the best performance by sequentially correcting previous errors.  
Feature engineering is applied within the pipeline to maintain consistency and avoid data leakage.


In [113]:
pre_gbr = ColumnTransformer([("cat", OneHotEncoder(**OHE_KW), selector(dtype_include=object))],
                            remainder="passthrough", verbose_feature_names_out=False)

pipe_gbr = Pipeline([
    ("feature_engineering", FeatureEngineeringTransformer_GBR()),
    ("preprocessor", pre_gbr),
    ("regressor", GradientBoostingRegressor(random_state=42))
])

param_gbr = {"regressor__n_estimators":[500,800],"regressor__learning_rate":[0.05,0.07],"regressor__max_depth":[4,5],
             "regressor__min_samples_split":[2,5],"regressor__min_samples_leaf":[1,3],"regressor__subsample":[0.8,0.9],
             "regressor__max_features":["sqrt",None]}
search = RandomizedSearchCV(pipe_gbr, param_distributions=param_gbr, n_iter=20, scoring="r2", cv=3, n_jobs=-1, random_state=42)
with parallel_backend("threading"):
    search.fit(X_train, y_train)
best_gbr = search.best_estimator_
cv_scores_gbr = cross_val_score(best_gbr, X_train, y_train, cv=cv5, scoring="r2", n_jobs=-1)

y_pred_gbr = best_gbr.predict(X_test)
R2_gbr, RMSE_gbr, MAE_gbr = r2_score(y_test, y_pred_gbr), rmse_func(y_test, y_pred_gbr), mean_absolute_error(y_test, y_pred_gbr)
fn_gbr = best_gbr.named_steps["preprocessor"].get_feature_names_out()
imp_gbr = pd.DataFrame({"Feature": fn_gbr, "Importance": best_gbr.named_steps["regressor"].feature_importances_}).sort_values("Importance", ascending=False).head(15)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


## Step 9: Summary of Model Performance
All metrics (Test R², CV R², RMSE, MAE) are compiled into a summary table for easy comparison.  
Higher R² and lower RMSE/MAE indicate better performance.  
This table forms the quantitative foundation for model comparison.


In [115]:
summary = pd.DataFrame([
    {"Model":"Linear Regression","Test R²":R2_mlr,"CV R² (5-fold)":np.mean(cv_scores_mlr),"RMSE":RMSE_mlr,"MAE":MAE_mlr},
    {"Model":"Decision Tree","Test R²":R2_dt,"CV R² (5-fold)":np.mean(cv_scores_dt),"RMSE":RMSE_dt,"MAE":MAE_dt},
    {"Model":"Gradient Boosting","Test R²":R2_gbr,"CV R² (5-fold)":np.mean(cv_scores_gbr),"RMSE":RMSE_gbr,"MAE":MAE_gbr}
])
print(summary.to_string(index=False))


            Model  Test R²  CV R² (5-fold)        RMSE         MAE
Linear Regression 0.738048        0.734188 3213.627102 2208.192837
    Decision Tree 0.779029        0.788007 2951.563093 1791.505570
Gradient Boosting 0.826125        0.836151 2618.202399 1624.783892


## Step 10: Model Comparison Charts
Interactive Plotly charts visualize:
- **RMSE and MAE Comparison**: Bar chart showing error magnitudes across models.
- **R² vs. CV R² Comparison**: Line chart comparing each model’s test and cross-validated performance.
These help interpret bias-variance tradeoffs and model robustness.


In [117]:
Result = summary.copy()

fig_error = go.Figure()
fig_error.add_trace(go.Bar(x=Result["Model"], y=Result["RMSE"], name="RMSE", marker_color="lightskyblue"))
fig_error.add_trace(go.Bar(x=Result["Model"], y=Result["MAE"], name="MAE", marker_color="lightgreen"))
fig_error.update_layout(title="RMSE & MAE Comparison Across Models", xaxis_title="Model", yaxis_title="Error Value", barmode="group", template="plotly_white")
fig_error.show()

fig_r2 = go.Figure()
fig_r2.add_trace(go.Scatter(x=Result["Model"], y=Result["Test R²"], name="Test R²", mode="lines+markers+text",
                            text=[f"{v:.3f}" for v in Result["Test R²"]], textposition="top center", line=dict(color="red", width=3)))
fig_r2.add_trace(go.Scatter(x=Result["Model"], y=Result["CV R² (5-fold)"], name="CV R² (5-fold)", mode="lines+markers+text",
                            text=[f"{v:.3f}" for v in Result["CV R² (5-fold)"]], textposition="top center", line=dict(color="royalblue", width=3, dash="dot")))
fig_r2.update_layout(title="R² Comparison (Test vs Cross-Validation)", xaxis_title="Model", yaxis_title="R² Score", yaxis=dict(range=[0,1]), template="plotly_white")
fig_r2.show()

## Step 11: Feature Importance Visualization

In [119]:
def plot_importances_interactive(df_imp, title):
    fig = px.bar(df_imp.sort_values("Importance", ascending=True), x="Importance", y="Feature", orientation="h", title=title, template="plotly_white")
    fig.update_layout(height=600)
    fig.show()

plot_importances_interactive(imp_mlr, "Top 15 Features — Linear Regression")
plot_importances_interactive(imp_dt,  "Top 15 Features — Decision Tree")
plot_importances_interactive(imp_gbr, "Top 15 Features — Gradient Boosting")

## Step 12: Residual Analysis
Residual plots (Actual − Predicted) for each model assess bias and model fit.  

In [121]:
def residual_plot_interactive(y_true, y_pred, title):
    res = y_true - y_pred
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=y_pred, y=res, mode="markers", name="Residuals"))
    fig.add_hline(y=0, line_dash="dash", line_color="red")
    fig.update_layout(title=title, xaxis_title="Predicted Price", yaxis_title="Residual (Actual - Predicted)", template="plotly_white", height=500)
    fig.show()

residual_plot_interactive(y_test, y_pred_mlr, "Residual Plot — Linear Regression")
residual_plot_interactive(y_test, y_pred_dt,  "Residual Plot — Decision Tree")
residual_plot_interactive(y_test, y_pred_gbr, "Residual Plot — Gradient Boosting")
